In [ ]:
import numpy as np
import tensorflow as tf
import pandas as pd
from sklearn.datasets import make_classification
from sklearn.model_selection import StratifiedShuffleSplit
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from matplotlib import pyplot as plt
from datetime import datetime

In [ ]:
def union_shuffled_copies(a, b):
    assert len(a) == len(b)
    p = np.random.permutation(len(a))
    return a[p], b[p]

def get_data(X, y_label, batch_size=1000):
    X_shuffled, y_label_shuffled = union_shuffled_copies(X, y_label)
    return X_shuffled[range(batch_size)], y_label_shuffled[range(batch_size)]

In [ ]:
raw = make_classification(n_samples=100000, n_features=8, n_informative=5, n_redundant=0, n_repeated=0, n_classes=2, n_clusters_per_class=1)
X = raw[0]
y = raw[1]

enc = OneHotEncoder()
y = enc.fit_transform(y.reshape(-1, 1)).toarray()

split = StratifiedShuffleSplit(n_splits=1, test_size=0.3)
for train_index, test_index in split.split(X, y):
    X_train = X[train_index]
    X_test = X[test_index]
    y_train = y[train_index]
    y_test = y[test_index]

print(X_train.shape, X_test.shape)
print(y_train.shape, y_test.shape)

In [ ]:
now = datetime.utcnow().strftime("%Y%m%d%H%M%S")
root_logdir = "tf_logs"
logdir = "{}/run-{}/".format(root_logdir, now)

In [ ]:
g = tf.Graph()
LEARNING_RATE = 0.05
EPOCHS = 100
BATCH_SIZE = 10000

with g.as_default():
    #build compute graph
    with tf.name_scope('inference') as scope:
        x = tf.placeholder(tf.float32, shape=[None, 8])
        y_true = tf.placeholder(tf.float32, shape=None)
        w1 = tf.Variable(np.random.randn(8, 5), dtype=tf.float32, name='weights1')
        b1 = tf.Variable(np.random.randn(1, 5), dtype=tf.float32, name='bias1')
        w2 = tf.Variable(np.random.randn(5, 2), dtype=tf.float32, name='weights2')
        b2 = tf.Variable(np.random.randn(1, 2), dtype=tf.float32, name='bias2')
        
#         layer1 = tf.sigmoid(tf.matmul(x, w1) + b1)
        layer1 = tf.nn.relu(tf.matmul(x, w1) + b1)
        layer2 = tf.matmul(layer1, w2) + b2
        y_pred = tf.nn.softmax(layer2)
        
        y_pred_01 = tf.argmax(y_pred, axis=1)
        y_true_01 = tf.argmax(y_true, axis=1)
        
    with tf.name_scope('loss') as scope:
        crossentropy_loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=layer2, labels=y_true))
        confusion_matrix = tf.confusion_matrix(labels=y_true_01, predictions=y_pred_01)
        
    with tf.name_scope('train') as scope:
        optimizer = tf.train.AdamOptimizer()
        train = optimizer.minimize(crossentropy_loss)
        saver = tf.train.Saver()
    
    with tf.name_scope('tensorboard') as scope:
        crossentropy_summary = tf.summary.scalar('crossentropy_loss', crossentropy_loss)
        file_writer = tf.summary.FileWriter(logdir, tf.get_default_graph())

    #initialize viriables
    init = tf.global_variables_initializer()
    
    #session
    with tf.Session() as sess:
        sess.run(init)
        
        for epoch in range(EPOCHS):
            for step in range(int(X_train.shape[0]/BATCH_SIZE)):
                X_per_step, y_per_step = get_data(X_train, y_train, batch_size=BATCH_SIZE)
                sess.run(train, {x: X_per_step, y_true: y_per_step})
            
            loss, loss_summary = sess.run(
                [crossentropy_loss, crossentropy_summary], 
                {x: X_test, y_true: y_test}
            )
            print("epoch {0}: {1}".format(epoch+1, loss))
            file_writer.add_summary(loss_summary, epoch)
            
        print("confusion matrix: {0}".format(sess.run(confusion_matrix, {x: X_test, y_true: y_test})))
        save_path = saver.save(sess, "../edy_model.ckpt")
        file_writer.close()

In [8]:
tf.reset_default_graph()